In [ ]:
import tensorflow as tf

import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
#!pip install datawig
#import datawig

from sklearn.model_selection import train_test_split
from sklearn.impute import KNNImputer
from sklearn.preprocessing import MinMaxScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split, cross_val_score
from imblearn.over_sampling import SMOTE

from pycaret.classification import *
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score

#!pip install pytorch_tabnet
import torch
from pytorch_tabnet.tab_model  import TabNetClassifier

#!pip install optuna
from sklearn.model_selection import cross_val_score
import optuna
import time

#!pip install tensorflow_addons == "0.22.0"
from tensorflow_addons.metrics import F1Score

## TabNet

In [ ]:
train = pd.read_csv("complete_train_duplication.csv")
submission = pd.read_csv("complete_submission_duplication.csv")

converted_mean = sum(train.is_converted)/len(train)

train, test = train_test_split(train, test_size = 0.4, shuffle = True, random_state = 12345)

original_train = pd.read_csv('conversion_train_etc_duplicate.csv')
original_sub = pd.read_csv('conversion_sub_etc_duplicate.csv')

original_test = original_train.iloc[test.index]
original_train = original_train.iloc[train.index]

########################### mean_customer_idx 컬럼 생성 ###########################################
original_train['customer_idx_cnt'] = original_train.groupby(['customer_idx'])['customer_idx'].transform('count')

#빈도 수 (customer_idx_cnt)가 5 이하인 데이터는 customer_idx를 etc로 통일
n = 5
for i in range(1,n+1):
    original_train.loc[original_train['customer_idx_cnt'] == i,'customer_idx'] = 'etc'

# customer_idx 별 평균 영업 성공률 계산한 mean_customer_idx 추가
original_train['mean_customer_idx'] = original_train.groupby('customer_idx')['is_converted'].transform('mean')

original_train.mean_customer_idx = [converted_mean if original_train.customer_idx.iloc[i] == "etc" else original_train.mean_customer_idx.iloc[i] for i in range(len(original_train))]
mean_customer_idx_unique = original_train.drop_duplicates(subset=['customer_idx'])[['customer_idx', 'mean_customer_idx']]

original_test = original_test.merge(mean_customer_idx_unique, on='customer_idx', how='left')
original_test['mean_customer_idx'].fillna(original_train[original_train['customer_idx']=='etc']['mean_customer_idx'].iloc[0],inplace= True)

original_sub = original_sub.merge(mean_customer_idx_unique, on='customer_idx', how='left')
original_sub['mean_customer_idx'].fillna(original_train[original_train['customer_idx']=='etc']['mean_customer_idx'].iloc[0],inplace= True)

##################### mean_lead_owner 컬럼 생성 ###################################
original_train['lead_owner_cnt'] = original_train.groupby(['lead_owner'])['is_converted'].transform('count')

#빈도 수 (lead_owner_cnt)가 5 이하인 데이터는 lead_owner를 etc로 통일
n = 5
for i in range(1,n+1):
    original_train.loc[original_train['lead_owner_cnt'] == i,'lead_owner'] = 'etc'

# lead_owner 별 평균 영업 성공률 계산한 mean_lead_owner 추가
original_train['mean_lead_owner'] = original_train.groupby('lead_owner')['is_converted'].transform('mean')
original_train.mean_lead_owner = [converted_mean if original_train.lead_owner.iloc[i] == "etc" else original_train.mean_lead_owner.iloc[i] for i in range(len(original_train))]
mean_customer_idx_unique = original_train.drop_duplicates(subset=['lead_owner'])[['lead_owner', 'mean_lead_owner']]

original_test = original_test.merge(mean_customer_idx_unique,on='lead_owner',how='left')
original_test['mean_lead_owner'].fillna(original_train[original_train['lead_owner']=='etc']['mean_lead_owner'].iloc[0],inplace= True)

original_sub = original_sub.merge(mean_customer_idx_unique,on='lead_owner',how='left')
original_sub['mean_lead_owner'].fillna(original_train[original_train['lead_owner']=='etc']['mean_lead_owner'].iloc[0],inplace= True)

##################### corporate_converted_ratio 컬럼 생성 ###################################
table = original_train.value_counts(["response_corporate","is_converted"]).reset_index().pivot(index = "is_converted", columns = "response_corporate", values = 0).T
table["Total"] = table.sum(axis = 1)
table["True_per"] = table[True]/table["Total"]
table = table.fillna(0)
table = table.reset_index()[["response_corporate", "True_per"]].set_index("response_corporate").T.to_dict()
original_train['corporate_converted_ratio'] = [table[i]["True_per"] if i in table.keys() else converted_mean for i in original_train.response_corporate]
original_test['corporate_converted_ratio'] = [table[i]["True_per"] if i in table.keys() else converted_mean for i in original_test.response_corporate]
original_sub['corporate_converted_ratio'] = [table[i]["True_per"] if i in table.keys() else converted_mean for i in original_sub.response_corporate]

original_train = original_train.reset_index(drop = True)
original_test = original_test.reset_index(drop = True)
train = train.reset_index(drop = True)
test = test.reset_index(drop = True)

train.mean_customer_idx = original_train.mean_customer_idx
test.mean_customer_idx = original_test.mean_customer_idx
submission.mean_customer_idx = original_sub.mean_customer_idx

train.mean_lead_owner = original_train.mean_lead_owner
test.mean_lead_owner = original_test.mean_lead_owner
submission.mean_lead_owner = original_sub.mean_lead_owner

train.corporate_converted_ratio = original_train.corporate_converted_ratio
test.corporate_converted_ratio = original_test.corporate_converted_ratio
submission.corporate_converted_ratio = original_sub.corporate_converted_ratio

train['inquiry_type'] = original_train.inquiry_type
test['inquiry_type'] = original_test.inquiry_type
submission['inquiry_type'] = original_sub.inquiry_type


### 학습
train = train.drop(["inquiry_type",'lead_desc_length','expected_timeline','ver_cus',"bant_submit","historical_existing_cnt","id_strategic_ver", "it_strategic_ver","response_corporate","customer_country", "customer_job","customer_position",'ver_win_rate_x','ver_win_ratio_per_bu', "inquiry_type_isna"], axis =1)
test = test.drop(["inquiry_type",'lead_desc_length','expected_timeline','ver_cus',"bant_submit","historical_existing_cnt","id_strategic_ver", "it_strategic_ver","response_corporate","customer_country", "customer_job","customer_position",'ver_win_rate_x','ver_win_ratio_per_bu', "inquiry_type_isna"], axis =1)
submission = submission.drop(["inquiry_type",'lead_desc_length','expected_timeline','ver_cus',"bant_submit","historical_existing_cnt","id_strategic_ver", "it_strategic_ver","response_corporate","customer_country", "customer_job","customer_position",'ver_win_rate_x','ver_win_ratio_per_bu', "inquiry_type_isna"], axis =1)

target = ["is_converted"]
categorical = [i for i in train.columns if train[i].dtype == "O"]
numeric = train.columns.drop(categorical + target).tolist()

x_train = train[train.columns.drop(target)]
y_train = train[target]

x_train["구분"] = "train"
test["구분"] = "test"
submission["구분"] = "sub"
new = pd.concat([x_train, test, submission]).reset_index(drop = True)

# 라벨 인코딩
c = []
cat_idxs = []
cat_dims = []
for i in range(x_train.shape[1]):
    if x_train.columns[i] in categorical and x_train.columns[i] != "is_converted" and x_train.columns[i] != "구분":
        encoder = LabelEncoder()
        new[x_train.columns[i]] = encoder.fit_transform(new[x_train.columns[i]])
        cat_idxs.append(i)
        cat_dims.append(len(encoder.classes_))
    
    
x_train = new[new.구분 == "train"].drop(["구분"], axis = 1).reset_index(drop = True)
test = new[new.구분 == "test"].drop(["구분"], axis = 1).reset_index(drop = True)
submission = new[new.구분 == "sub"].drop(["구분"], axis = 1).reset_index(drop = True)

x_train = x_train.drop(["id", "is_converted"], axis = 1)
test = test.drop(["id"], axis = 1)
submission = submission.drop(target, axis = 1)


scaler = MinMaxScaler()
t = scaler.fit_transform(x_train[categorical+numeric])
x_train[categorical+numeric] = pd.DataFrame(t, columns = categorical+numeric)

t = scaler.transform(test[categorical+numeric])
test[categorical+numeric] = pd.DataFrame(t, columns = categorical+numeric)


t = scaler.transform(submission[categorical+numeric])
submission[categorical+numeric] = pd.DataFrame(t, columns = categorical+numeric)

x_test = test.drop(target, axis = 1)
y_test = test[target]

x_test, x_val, y_test, y_val = train_test_split(x_test, y_test, test_size = 0.5, random_state = 42, shuffle = True)

smote = SMOTE(random_state=42)
x_train, y_train = smote.fit_resample(x_train, y_train)

def objective(trial):
    cat_emb_dim = trial.suggest_int('cat_emb_dim', 3, 10)
    lr = trial.suggest_loguniform('lr', 0.001, 0.0017)
    step_size = trial.suggest_int('step_size', 30, 40)
    gamma = trial.suggest_uniform('gamma', 0.6, 0.8)
    patience = trial.suggest_int('patience', 2, 5)
    batch_size = trial.suggest_categorical('batch_size', [32, 64, 128, 256])
    virtual_batch_size = trial.suggest_categorical('virtual_batch_size', [32, 64, 128])
    weights = trial.suggest_categorical('weights', [0,1])
    
    clf = TabNetClassifier(cat_idxs=cat_idxs,
                       cat_dims=cat_dims,
                       cat_emb_dim=cat_emb_dim,
                       optimizer_fn=torch.optim.Adam,
                       optimizer_params=dict(lr=lr),
                       scheduler_params={"step_size":step_size,
                                         "gamma":gamma},
                       scheduler_fn=torch.optim.lr_scheduler.StepLR,
                       mask_type='entmax' # "sparsemax", entmax
                      )
    
    clf.fit(
        X_train=x_train.values, y_train=y_train.values.reshape(-1)*1,
        eval_set=[(x_train.values, y_train.values.reshape(-1)*1),(x_val.values, np.asarray(y_val.values.reshape(-1)*1, dtype = int))],
        eval_name=["train", 'valid'],
        eval_metric=["accuracy"],
        max_epochs=30 , patience=patience,
        batch_size=batch_size, virtual_batch_size=virtual_batch_size,
        num_workers=1,
        weights=weights,
        drop_last=False,
    )
    
    preds = clf.predict(x_test.values)
    
    f1 = f1_score(preds, np.asarray(y_test.values.reshape(-1)*1, dtype = int))
    return f1


study = optuna.create_study(direction='maximize')

start = time.time()
study.optimize(objective, n_trials=50)
end = time.time()
print(end - start)

In [ ]:
train = pd.read_csv("complete_train_duplication.csv")
submission = pd.read_csv("complete_submission_duplication.csv")

converted_mean = sum(train.is_converted)/len(train)

train, test = train_test_split(train, test_size = 0.4, shuffle = True, random_state = 12345)

original_train = pd.read_csv('conversion_train_etc_duplicate.csv')
original_sub = pd.read_csv('conversion_sub_etc_duplicate.csv')

original_test = original_train.iloc[test.index]
original_train = original_train.iloc[train.index]

########################### mean_customer_idx 컬럼 생성 ###########################################
original_train['customer_idx_cnt'] = original_train.groupby(['customer_idx'])['customer_idx'].transform('count')

#빈도 수 (customer_idx_cnt)가 5 이하인 데이터는 customer_idx를 etc로 통일
n = 5
for i in range(1,n+1):
    original_train.loc[original_train['customer_idx_cnt'] == i,'customer_idx'] = 'etc'

# customer_idx 별 평균 영업 성공률 계산한 mean_customer_idx 추가
original_train['mean_customer_idx'] = original_train.groupby('customer_idx')['is_converted'].transform('mean')

original_train.mean_customer_idx = [converted_mean if original_train.customer_idx.iloc[i] == "etc" else original_train.mean_customer_idx.iloc[i] for i in range(len(original_train))]
mean_customer_idx_unique = original_train.drop_duplicates(subset=['customer_idx'])[['customer_idx', 'mean_customer_idx']]

original_test = original_test.merge(mean_customer_idx_unique, on='customer_idx', how='left')
original_test['mean_customer_idx'].fillna(original_train[original_train['customer_idx']=='etc']['mean_customer_idx'].iloc[0],inplace= True)

original_sub = original_sub.merge(mean_customer_idx_unique, on='customer_idx', how='left')
original_sub['mean_customer_idx'].fillna(original_train[original_train['customer_idx']=='etc']['mean_customer_idx'].iloc[0],inplace= True)

##################### mean_lead_owner 컬럼 생성 ###################################
original_train['lead_owner_cnt'] = original_train.groupby(['lead_owner'])['is_converted'].transform('count')

#빈도 수 (lead_owner_cnt)가 5 이하인 데이터는 lead_owner를 etc로 통일
n = 5
for i in range(1,n+1):
    original_train.loc[original_train['lead_owner_cnt'] == i,'lead_owner'] = 'etc'

# lead_owner 별 평균 영업 성공률 계산한 mean_lead_owner 추가
original_train['mean_lead_owner'] = original_train.groupby('lead_owner')['is_converted'].transform('mean')
original_train.mean_lead_owner = [converted_mean if original_train.lead_owner.iloc[i] == "etc" else original_train.mean_lead_owner.iloc[i] for i in range(len(original_train))]
mean_customer_idx_unique = original_train.drop_duplicates(subset=['lead_owner'])[['lead_owner', 'mean_lead_owner']]

original_test = original_test.merge(mean_customer_idx_unique,on='lead_owner',how='left')
original_test['mean_lead_owner'].fillna(original_train[original_train['lead_owner']=='etc']['mean_lead_owner'].iloc[0],inplace= True)

original_sub = original_sub.merge(mean_customer_idx_unique,on='lead_owner',how='left')
original_sub['mean_lead_owner'].fillna(original_train[original_train['lead_owner']=='etc']['mean_lead_owner'].iloc[0],inplace= True)

##################### corporate_converted_ratio 컬럼 생성 ###################################
table = original_train.value_counts(["response_corporate","is_converted"]).reset_index().pivot(index = "is_converted", columns = "response_corporate", values = 0).T
table["Total"] = table.sum(axis = 1)
table["True_per"] = table[True]/table["Total"]
table = table.fillna(0)
table = table.reset_index()[["response_corporate", "True_per"]].set_index("response_corporate").T.to_dict()
original_train['corporate_converted_ratio'] = [table[i]["True_per"] if i in table.keys() else converted_mean for i in original_train.response_corporate]
original_test['corporate_converted_ratio'] = [table[i]["True_per"] if i in table.keys() else converted_mean for i in original_test.response_corporate]
original_sub['corporate_converted_ratio'] = [table[i]["True_per"] if i in table.keys() else converted_mean for i in original_sub.response_corporate]

original_train = original_train.reset_index(drop = True)
original_test = original_test.reset_index(drop = True)
train = train.reset_index(drop = True)
test = test.reset_index(drop = True)

train.mean_customer_idx = original_train.mean_customer_idx
test.mean_customer_idx = original_test.mean_customer_idx
submission.mean_customer_idx = original_sub.mean_customer_idx

train.mean_lead_owner = original_train.mean_lead_owner
test.mean_lead_owner = original_test.mean_lead_owner
submission.mean_lead_owner = original_sub.mean_lead_owner

train.corporate_converted_ratio = original_train.corporate_converted_ratio
test.corporate_converted_ratio = original_test.corporate_converted_ratio
submission.corporate_converted_ratio = original_sub.corporate_converted_ratio


### 학습
train = train.drop(['lead_desc_length','expected_timeline','ver_cus',"bant_submit","historical_existing_cnt","id_strategic_ver", "it_strategic_ver","response_corporate","customer_country", "customer_job","customer_position",'ver_win_rate_x','ver_win_ratio_per_bu', "inquiry_type_isna"], axis =1)
test = test.drop(['lead_desc_length','expected_timeline','ver_cus',"bant_submit","historical_existing_cnt","id_strategic_ver", "it_strategic_ver","response_corporate","customer_country", "customer_job","customer_position",'ver_win_rate_x','ver_win_ratio_per_bu', "inquiry_type_isna"], axis =1)
submission = submission.drop(['lead_desc_length','expected_timeline','ver_cus',"bant_submit","historical_existing_cnt","id_strategic_ver", "it_strategic_ver","response_corporate","customer_country", "customer_job","customer_position",'ver_win_rate_x','ver_win_ratio_per_bu', "inquiry_type_isna"], axis =1)

target = ["is_converted"]
categorical = [i for i in train.columns if train[i].dtype == "O"]
numeric = train.columns.drop(categorical + target).tolist()

x_train = train[train.columns.drop(target)]
y_train = train[target]

x_train["구분"] = "train"
test["구분"] = "test"
submission["구분"] = "sub"
new = pd.concat([x_train, test, submission]).reset_index(drop = True)

# 라벨 인코딩
c = []
cat_idxs = []
cat_dims = []
for i in range(x_train.shape[1]):
    if x_train.columns[i] in categorical and x_train.columns[i] != "is_converted" and x_train.columns[i] != "구분":
        encoder = LabelEncoder()
        new[x_train.columns[i]] = encoder.fit_transform(new[x_train.columns[i]])
        cat_idxs.append(i)
        cat_dims.append(len(encoder.classes_))
    
    
x_train = new[new.구분 == "train"].drop(["구분"], axis = 1).reset_index(drop = True)
test = new[new.구분 == "test"].drop(["구분"], axis = 1).reset_index(drop = True)
submission = new[new.구분 == "sub"].drop(["구분"], axis = 1).reset_index(drop = True)

x_train = x_train.drop(["id", "is_converted"], axis = 1)
test = test.drop(["id"], axis = 1)
submission = submission.drop(target, axis = 1)


scaler = MinMaxScaler()
t = scaler.fit_transform(x_train[categorical+numeric])
x_train[categorical+numeric] = pd.DataFrame(t, columns = categorical+numeric)

t = scaler.transform(test[categorical+numeric])
test[categorical+numeric] = pd.DataFrame(t, columns = categorical+numeric)


t = scaler.transform(submission[categorical+numeric])
submission[categorical+numeric] = pd.DataFrame(t, columns = categorical+numeric)

x_test = test.drop(target, axis = 1)
y_test = test[target]

x_test, x_val, y_test, y_val = train_test_split(x_test, y_test, test_size = 0.5, random_state = 42, shuffle = True)

smote = SMOTE(random_state=42)
x_train, y_train = smote.fit_resample(x_train, y_train)

best = study.best_trial.params
# 베스트 스코어로 적합 시켜보기
clf = TabNetClassifier(cat_idxs=cat_idxs,
                       cat_dims=cat_dims,
                       cat_emb_dim=best["cat_emb_dim"],
                       optimizer_fn=torch.optim.Adam,
                       optimizer_params=dict(lr=best["lr"]),
                       scheduler_params={"step_size":best["step_size"],
                                         "gamma":best["gamma"]},
                       scheduler_fn=torch.optim.lr_scheduler.StepLR,
                       mask_type='entmax' # "sparsemax", entmax
                      )

clf.fit(
    X_train=x_train.values, y_train=y_train.values.reshape(-1)*1,
    eval_set=[(x_train.values, y_train.values.reshape(-1)*1), (x_val.values, np.asarray(y_val.values.reshape(-1)*1, dtype = int))],
    eval_name=['train','valid'],
    eval_metric=["accuracy"],
    max_epochs=20 , patience=best["patience"],
    batch_size=best["batch_size"], virtual_batch_size=best["virtual_batch_size"],
    num_workers=0,
    weights=1,
    drop_last=False,
)

preds = clf.predict(x_test.values)
f1_score(preds, np.asarray(y_test.values.reshape(-1)*1, dtype = int))

In [ ]:
train = pd.read_csv("complete_train_duplication.csv")
submission = pd.read_csv("complete_submission_duplication.csv")

converted_mean = sum(train.is_converted)/len(train)

original_train = pd.read_csv('conversion_train_etc_duplicate.csv')
original_sub = pd.read_csv('conversion_sub_etc_duplicate.csv')

########################### mean_customer_idx 컬럼 생성 ###########################################
original_train['customer_idx_cnt'] = original_train.groupby(['customer_idx'])['customer_idx'].transform('count')

#빈도 수 (customer_idx_cnt)가 5 이하인 데이터는 customer_idx를 etc로 통일
n = 5
for i in range(1,n+1):
    original_train.loc[original_train['customer_idx_cnt'] == i,'customer_idx'] = 'etc'

# customer_idx 별 평균 영업 성공률 계산한 mean_customer_idx 추가
original_train['mean_customer_idx'] = original_train.groupby('customer_idx')['is_converted'].transform('mean')

original_train.mean_customer_idx = [converted_mean if original_train.customer_idx.iloc[i] == "etc" else original_train.mean_customer_idx.iloc[i] for i in range(len(original_train))]
mean_customer_idx_unique = original_train.drop_duplicates(subset=['customer_idx'])[['customer_idx', 'mean_customer_idx']]

original_sub = original_sub.merge(mean_customer_idx_unique, on='customer_idx', how='left')
original_sub['mean_customer_idx'].fillna(original_train[original_train['customer_idx']=='etc']['mean_customer_idx'].iloc[0],inplace= True)

##################### mean_lead_owner 컬럼 생성 ###################################
original_train['lead_owner_cnt'] = original_train.groupby(['lead_owner'])['is_converted'].transform('count')

#빈도 수 (lead_owner_cnt)가 5 이하인 데이터는 lead_owner를 etc로 통일
n = 5
for i in range(1,n+1):
    original_train.loc[original_train['lead_owner_cnt'] == i,'lead_owner'] = 'etc'

# lead_owner 별 평균 영업 성공률 계산한 mean_lead_owner 추가
original_train['mean_lead_owner'] = original_train.groupby('lead_owner')['is_converted'].transform('mean')
original_train.mean_lead_owner = [converted_mean if original_train.lead_owner.iloc[i] == "etc" else original_train.mean_lead_owner.iloc[i] for i in range(len(original_train))]
mean_customer_idx_unique = original_train.drop_duplicates(subset=['lead_owner'])[['lead_owner', 'mean_lead_owner']]

original_sub = original_sub.merge(mean_customer_idx_unique,on='lead_owner',how='left')
original_sub['mean_lead_owner'].fillna(original_train[original_train['lead_owner']=='etc']['mean_lead_owner'].iloc[0],inplace= True)

##################### corporate_converted_ratio 컬럼 생성 ###################################
table = original_train.value_counts(["response_corporate","is_converted"]).reset_index().pivot(index = "is_converted", columns = "response_corporate", values = 0).T
table["Total"] = table.sum(axis = 1)
table["True_per"] = table[True]/table["Total"]
table = table.fillna(0)
table = table.reset_index()[["response_corporate", "True_per"]].set_index("response_corporate").T.to_dict()
original_train['corporate_converted_ratio'] = [table[i]["True_per"] if i in table.keys() else converted_mean for i in original_train.response_corporate]
original_sub['corporate_converted_ratio'] = [table[i]["True_per"] if i in table.keys() else converted_mean for i in original_sub.response_corporate]

train.mean_customer_idx = original_train.mean_customer_idx
submission.mean_customer_idx = original_sub.mean_customer_idx

train.mean_lead_owner = original_train.mean_lead_owner
submission.mean_lead_owner = original_sub.mean_lead_owner

train.corporate_converted_ratio = original_train.corporate_converted_ratio
submission.corporate_converted_ratio = original_sub.corporate_converted_ratio

### 학습
train = train.drop(['lead_desc_length','expected_timeline','ver_cus',"bant_submit","historical_existing_cnt","id_strategic_ver", "it_strategic_ver","response_corporate","customer_country", "customer_job","customer_position",'ver_win_rate_x','ver_win_ratio_per_bu', "inquiry_type_isna"], axis =1)
submission = submission.drop(['lead_desc_length','expected_timeline','ver_cus',"bant_submit","historical_existing_cnt","id_strategic_ver", "it_strategic_ver","response_corporate","customer_country", "customer_job","customer_position",'ver_win_rate_x','ver_win_ratio_per_bu', "inquiry_type_isna"], axis =1)

target = ["is_converted"]
categorical = [i for i in train.columns if train[i].dtype == "O"]
numeric = train.columns.drop(categorical + target).tolist()

train["구분"] = "train"
submission["구분"] = "sub"
new = pd.concat([train, submission]).reset_index(drop = True)

# 라벨 인코딩
c = []
cat_idxs = []
cat_dims = []
for i in range(train.shape[1]):
    if train.columns[i] in categorical and train.columns[i] != "is_converted" and train.columns[i] != "구분":
        encoder = LabelEncoder()
        new[train.columns[i]] = encoder.fit_transform(new[train.columns[i]])
        cat_idxs.append(i)
        cat_dims.append(len(encoder.classes_))
    
    
train = new[new.구분 == "train"].drop(["구분"], axis = 1).reset_index(drop = True)
submission = new[new.구분 == "sub"].drop(["구분"], axis = 1).reset_index(drop = True)

train = train.drop(["id"], axis = 1)


scaler = MinMaxScaler()
t = scaler.fit_transform(train[categorical+numeric])
train[categorical+numeric] = pd.DataFrame(t, columns = categorical+numeric)

t = scaler.transform(submission[categorical+numeric])
submission[categorical+numeric] = pd.DataFrame(t, columns = categorical+numeric)

X = train[train.columns.drop(target)]
y = train[target]

x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42, shuffle = True)

smote = SMOTE(random_state=42)
x_train, y_train = smote.fit_resample(x_train, y_train)

best = {'cat_emb_dim': 6,
 'lr': 0.0010841620912501022,
 'step_size': 36,
 'gamma': 0.6202850279587523,
 'patience': 21,
 'batch_size': 32,
 'virtual_batch_size': 64,
 'weights': 0}

clf = TabNetClassifier(cat_idxs=cat_idxs,
                       cat_dims=cat_dims,
                       cat_emb_dim=best["cat_emb_dim"],
                       optimizer_fn=torch.optim.Adam,
                       optimizer_params=dict(lr=best["lr"]),
                       scheduler_params={"step_size":best["step_size"],
                                         "gamma":best["gamma"]},
                       scheduler_fn=torch.optim.lr_scheduler.StepLR,
                       mask_type='entmax' # "sparsemax", entmax
                      )

clf.fit(
    X_train=x_train.values, y_train=y_train.values.reshape(-1)*1,
    eval_set=[(x_train.values, y_train.values.reshape(-1)*1), (x_test.values, y_test.values.reshape(-1)*1)],
    eval_name=['train','test'],
    eval_metric=["accuracy"],
    max_epochs=5 , patience=best["patience"],
    batch_size=best["batch_size"], virtual_batch_size=best["virtual_batch_size"],
    num_workers=0,
    weights=1,
    drop_last=False,
)

preds = clf.predict(submission.drop(["id", "is_converted"], axis = 1).values)

submission = pd.read_csv("submission.csv")
submission['is_converted'] = preds
submission.to_csv("submission.csv", index=False)

## MLP

In [ ]:
# 모든 전처리가 완료된 데이터 (모델 돌릴 때마다 전처리하고 싶지 않아서 csv파일로 저장해둔 것)
train = pd.read_csv("complete_train_duplication.csv")
submission = pd.read_csv("complete_submission_duplication.csv")

# 전체 평균
converted_mean = sum(train.is_converted)/len(train)

# train, test로 미리 나눔
# -> 영업 성공률이 쓰이는 변수는 분리를 해서 train으로 변수를 만들고 그에 따라 test에 집어넣어야 실제 제출하는 submission이랑 비슷하다고 판단
train, test = train_test_split(train, test_size = 0.4, shuffle = True, random_state = 12345)

# 영업 성공률이 쓰이는 세 변수를 재생성
# -> 빈도수가 5 이하거나 결측치인 경우 기존에는 etc로 묶어 etc의 영업성공률 평균으로 계산
# => 수정 버전은 etc의 영업성공률이 아닌 전체 train의 영업 성공률로 변환

# 변수 재생성을 위해 전처리가 되지 않은 파일 read (중복행 제거 및 customer_country만 변경된 데이터)
original_train = pd.read_csv('conversion_train_etc_duplicate.csv')
original_sub = pd.read_csv('conversion_sub_etc_duplicate.csv')

# 위 train, test로 나눈데로 index를 이용해 똑같이 나눔
original_test = original_train.iloc[test.index]
original_train = original_train.iloc[train.index]

########################### mean_customer_idx 컬럼 생성 ###########################################
original_train['customer_idx_cnt'] = original_train.groupby(['customer_idx'])['customer_idx'].transform('count')

#빈도 수 (customer_idx_cnt)가 5 이하인 데이터는 customer_idx를 etc로 통일
n = 5
for i in range(1,n+1):
    original_train.loc[original_train['customer_idx_cnt'] == i,'customer_idx'] = 'etc'

# customer_idx 별 평균 영업 성공률 계산한 mean_customer_idx 추가
original_train['mean_customer_idx'] = original_train.groupby('customer_idx')['is_converted'].transform('mean')

original_train.mean_customer_idx = [converted_mean if original_train.customer_idx.iloc[i] == "etc" else original_train.mean_customer_idx.iloc[i] for i in range(len(original_train))]
mean_customer_idx_unique = original_train.drop_duplicates(subset=['customer_idx'])[['customer_idx', 'mean_customer_idx']]

# train을 기준으로 test와 submission에 적용
original_test = original_test.merge(mean_customer_idx_unique, on='customer_idx', how='left')
original_test['mean_customer_idx'].fillna(original_train[original_train['customer_idx']=='etc']['mean_customer_idx'].iloc[0],inplace= True)

original_sub = original_sub.merge(mean_customer_idx_unique, on='customer_idx', how='left')
original_sub['mean_customer_idx'].fillna(original_train[original_train['customer_idx']=='etc']['mean_customer_idx'].iloc[0],inplace= True)

##################### mean_lead_owner 컬럼 생성 ###################################
original_train['lead_owner_cnt'] = original_train.groupby(['lead_owner'])['is_converted'].transform('count')

#빈도 수 (lead_owner_cnt)가 5 이하인 데이터는 lead_owner를 etc로 통일
n = 5
for i in range(1,n+1):
    original_train.loc[original_train['lead_owner_cnt'] == i,'lead_owner'] = 'etc'

# lead_owner 별 평균 영업 성공률 계산한 mean_lead_owner 추가
original_train['mean_lead_owner'] = original_train.groupby('lead_owner')['is_converted'].transform('mean')
original_train.mean_lead_owner = [converted_mean if original_train.lead_owner.iloc[i] == "etc" else original_train.mean_lead_owner.iloc[i] for i in range(len(original_train))]
mean_customer_idx_unique = original_train.drop_duplicates(subset=['lead_owner'])[['lead_owner', 'mean_lead_owner']]

# train을 기준으로 test와 submission에 적용
original_test = original_test.merge(mean_customer_idx_unique,on='lead_owner',how='left')
original_test['mean_lead_owner'].fillna(original_train[original_train['lead_owner']=='etc']['mean_lead_owner'].iloc[0],inplace= True)

original_sub = original_sub.merge(mean_customer_idx_unique,on='lead_owner',how='left')
original_sub['mean_lead_owner'].fillna(original_train[original_train['lead_owner']=='etc']['mean_lead_owner'].iloc[0],inplace= True)

##################### corporate_converted_ratio 컬럼 생성 ###################################
table = original_train.value_counts(["response_corporate","is_converted"]).reset_index().pivot(index = "is_converted", columns = "response_corporate", values = 0).T
table["Total"] = table.sum(axis = 1)
table["True_per"] = table[True]/table["Total"]
table = table.fillna(0)
table = table.reset_index()[["response_corporate", "True_per"]].set_index("response_corporate").T.to_dict()
original_train['corporate_converted_ratio'] = [table[i]["True_per"] if i in table.keys() else converted_mean for i in original_train.response_corporate]

# train을 기준으로 test와 submission에 적용
original_test['corporate_converted_ratio'] = [table[i]["True_per"] if i in table.keys() else converted_mean for i in original_test.response_corporate]
original_sub['corporate_converted_ratio'] = [table[i]["True_per"] if i in table.keys() else converted_mean for i in original_sub.response_corporate]

# index 재정렬
original_train = original_train.reset_index(drop = True)
original_test = original_test.reset_index(drop = True)
train = train.reset_index(drop = True)
test = test.reset_index(drop = True)

# 바꾼 세 변수를 train, test, submission에 각각 적용
train.mean_customer_idx = original_train.mean_customer_idx
test.mean_customer_idx = original_test.mean_customer_idx
submission.mean_customer_idx = original_sub.mean_customer_idx

train.mean_lead_owner = original_train.mean_lead_owner
test.mean_lead_owner = original_test.mean_lead_owner
submission.mean_lead_owner = original_sub.mean_lead_owner

train.corporate_converted_ratio = original_train.corporate_converted_ratio
test.corporate_converted_ratio = original_test.corporate_converted_ratio
submission.corporate_converted_ratio = original_sub.corporate_converted_ratio

######## 학습
# 불필요한 변수 drop
train = train.drop(['lead_desc_length','expected_timeline','ver_cus',"bant_submit","historical_existing_cnt","id_strategic_ver", "it_strategic_ver","response_corporate","customer_country", "customer_job","customer_position",'ver_win_rate_x','ver_win_ratio_per_bu', "inquiry_type_isna"], axis =1)
test = test.drop(['lead_desc_length','expected_timeline','ver_cus',"bant_submit","historical_existing_cnt","id_strategic_ver", "it_strategic_ver","response_corporate","customer_country", "customer_job","customer_position",'ver_win_rate_x','ver_win_ratio_per_bu', "inquiry_type_isna"], axis =1)
submission = submission.drop(['lead_desc_length','expected_timeline','ver_cus',"bant_submit","historical_existing_cnt","id_strategic_ver", "it_strategic_ver","response_corporate","customer_country", "customer_job","customer_position",'ver_win_rate_x','ver_win_ratio_per_bu', "inquiry_type_isna"], axis =1)

# 범주형, 수치형 변수로 나눔
target = ["is_converted"]
categorical = [i for i in train.columns if train[i].dtype == "O"]
numeric = train.columns.drop(categorical + target).tolist()

train["구분"] = "train"
test["구분"] = "test"
submission["구분"] = "sub"

new = pd.concat([train, test, submission]).reset_index(drop = True)

# 인코딩
c = []
for i in categorical:
    encoder = OneHotEncoder(drop='first')
    encoder.fit(new[[i]])
    new_df = encoder.transform(new[[i]])
    
    new_df = pd.DataFrame(new_df.toarray(), columns = encoder.get_feature_names_out([i]))
    new = new.drop([i], axis = 1)
    new = pd.concat([new, new_df], axis = 1)
    c += encoder.get_feature_names_out([i]).tolist()

categorical = c
train = new[new.구분 == "train"].drop(["구분"], axis = 1).reset_index(drop = True)
test = new[new.구분 == "test"].drop(["구분"], axis = 1).reset_index(drop = True)
submission = new[new.구분 == "sub"].drop(["구분"], axis = 1).reset_index(drop = True)

train = train.drop(["id"], axis = 1)
test = test.drop(["id"], axis = 1)
    

# 결측치 채우기
#imputer = KNNImputer(n_neighbors = 3)
#t = imputer.fit_transform(new_train[categorical + numeric])
#new_train[categorical + numeric] = pd.DataFrame(t, columns = categorical + numeric)

# MinMax scaling
scaler = MinMaxScaler()
t = scaler.fit_transform(train[categorical+numeric])
train[categorical+numeric] = pd.DataFrame(t, columns = categorical+numeric)

t = scaler.transform(test[categorical+numeric])
test[categorical+numeric] = pd.DataFrame(t, columns = categorical+numeric)

t = scaler.transform(submission[categorical+numeric])
submission[categorical+numeric] = pd.DataFrame(t, columns = categorical+numeric)

# train데이터를 x, y로 나눔
x_train = train[train.columns.drop(target)]
y_train = train[target]

# test 데이터를 x, y로 나눔
x_test = test[test.columns.drop(target)]
y_test = test[target]

# test 데이터에서 validation 데이터를 생성 -> 영업 성공률 변수의 영향을 받지 않기 위해서
x_test, x_val, y_test, y_val = train_test_split(x_test, y_test, test_size = 0.2, random_state = 42, shuffle = True)

# train만 SMOTE
smote = SMOTE(random_state=42)
x_train, y_train = smote.fit_resample(x_train, y_train)


In [ ]:
Parameter_loss = pd.DataFrame([], columns = ["Parameters", "Loss"])
def objective(trial):
    # 최적화 시킬 파라미터 후보들
    params = []
    patience = trial.suggest_int('patience', 1, 3)
    min_delta = trial.suggest_categorical('min_delta', [0.004, 0.005, 0.006, 0.007, 0.008])

    node1 = trial.suggest_categorical('node1', [16, 32, 64, 128])
    dropout1 = trial.suggest_uniform('dropout1', 0, 1)
    node2 = trial.suggest_categorical('node2', [16, 32, 64, 128])
    dropout2 = trial.suggest_uniform('dropout2', 0, 1)
    num_layers = trial.suggest_categorical('num_layers', ["two", "three"])
    if num_layers != "two":
            node3 = trial.suggest_categorical('node3', [16, 32, 64, 128])
            dropout3 = trial.suggest_uniform('dropout3', 0, 1)
    lr = trial.suggest_uniform('lr', 0.0001, 0.025)
    
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor = 'val_f1_score',
                                                      mode = 'min',
                                                      patience = patience,
                                                      min_delta = min_delta)

    # MLP 코드
    tf.random.set_seed(89)
    initializer = tf.keras.initializers.HeNormal(seed = 89)
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Dense(node1, 
                                    activation = 'relu', kernel_initializer=initializer))
    model.add(tf.keras.layers.Dropout(dropout1))
    if num_layers == "two":
        model.add(tf.keras.layers.Dense(node2,  
                                        activation = 'relu', kernel_initializer=initializer))
        model.add(tf.keras.layers.Dropout(dropout2))
    else:
        model.add(tf.keras.layers.Dense(node2, 
                                        activation = 'relu', kernel_initializer=initializer))
        model.add(tf.keras.layers.Dropout(dropout2))

        model.add(tf.keras.layers.Dense(node3, 
                                        activation = 'relu', kernel_initializer=initializer))
        model.add(tf.keras.layers.Dropout(dropout3))
    model.add(tf.keras.layers.Dense(2, activation = 'softmax', kernel_initializer=initializer))
    
    model.compile(loss = tf.keras.losses.BinaryCrossentropy(),
                      optimizer = tf.keras.optimizers.Adam(learning_rate = lr),
                      metrics = F1Score(num_classes = 2))
    
    # Y값을 원핫 인코딩하여 구성
    y_train_one_hot = pd.get_dummies(y_train.is_converted).values.reshape(-1,2)
    y_test_one_hot = pd.get_dummies(y_test.is_converted).values.reshape(-1,2)
    y_val_one_hot =pd.get_dummies( y_val.is_converted).values.reshape(-1,2)
    
    history = model.fit(np.array(x_train).reshape(-1,x_train.shape[1]), y_train_one_hot,
                             epochs = 30,
                             validation_data = [np.array(x_val).reshape(-1,x_train.shape[1]), y_val_one_hot],
                            verbose = 2)
    error = model.evaluate(x_test, y_test_one_hot)
    
    if num_layers == "two":
        params = [patience, min_delta, node1, dropout1,node2, dropout2, lr]
    else:
        params = [patience, min_delta, node1, dropout1,node2, dropout2,node3, dropout3, lr]
        
    Parameter_loss.loc[len(Parameter_loss)] = [params,error[1][1]]
    pred = model.predict(x_test)
    f1 = f1_score(pred[:,1] > 0.5, y_test.is_converted.values)
    return f1


study = optuna.create_study(direction='maximize')
start = time.time()
study.optimize(objective, n_trials=50)
end = time.time()
print(end - start)

In [ ]:
best = study.best_trial.params
# best 파라미터로 확인
early_stopping = tf.keras.callbacks.EarlyStopping(monitor = 'val_f1_score',
                                                      mode = 'min',
                                                      patience = best["patience"],
                                                      min_delta = best["min_delta"])


tf.random.set_seed(42)
initializer = tf.keras.initializers.HeNormal(seed = 42)
    
model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(best["node1"],activation = 'relu', kernel_initializer=initializer))
model.add(tf.keras.layers.Dropout(best["dropout1"]))
model.add(tf.keras.layers.Dense(best["node2"], activation = 'relu', kernel_initializer=initializer))
model.add(tf.keras.layers.Dropout(best["dropout2"]))
model.add(tf.keras.layers.Dense(best["node3"], activation = 'relu', kernel_initializer=initializer))
model.add(tf.keras.layers.Dropout(best["dropout3"]))
model.add(tf.keras.layers.Dense(2, activation = 'softmax', kernel_initializer=initializer))
    
model.compile(loss = tf.keras.losses.BinaryCrossentropy(),
                      optimizer = tf.keras.optimizers.Adam(learning_rate = best["lr"]),
                      metrics = F1Score(num_classes = 2))
    
y_train_one_hot = pd.get_dummies(y_train.is_converted).values.reshape(-1,2)
y_test_one_hot = pd.get_dummies(y_test.is_converted).values.reshape(-1,2)
y_val_one_hot =pd.get_dummies( y_val.is_converted).values.reshape(-1,2)
    
history = model.fit(np.array(x_train).reshape(-1,x_train.shape[1]), y_train_one_hot,
                             epochs = 30,
                             validation_data = [np.array(x_val).reshape(-1,x_train.shape[1]), y_val_one_hot],
                            verbose = 2)

In [ ]:
preds = model.predict(x_test.values)
f1_score(preds[:,1] > 0.5, y_test.values.reshape(-1))

In [ ]:
train = pd.read_csv("complete_train_duplication.csv")
submission = pd.read_csv("complete_submission_duplication.csv")

converted_mean = sum(train.is_converted)/len(train)

original_train = pd.read_csv('conversion_train_etc_duplicate.csv')
original_sub = pd.read_csv('conversion_sub_etc_duplicate.csv')

########################### mean_customer_idx 컬럼 생성 ###########################################
original_train['customer_idx_cnt'] = original_train.groupby(['customer_idx'])['customer_idx'].transform('count')

#빈도 수 (customer_idx_cnt)가 5 이하인 데이터는 customer_idx를 etc로 통일
n = 5
for i in range(1,n+1):
    original_train.loc[original_train['customer_idx_cnt'] == i,'customer_idx'] = 'etc'

# customer_idx 별 평균 영업 성공률 계산한 mean_customer_idx 추가
original_train['mean_customer_idx'] = original_train.groupby('customer_idx')['is_converted'].transform('mean')

original_train.mean_customer_idx = [converted_mean if original_train.customer_idx.iloc[i] == "etc" else original_train.mean_customer_idx.iloc[i] for i in range(len(original_train))]
mean_customer_idx_unique = original_train.drop_duplicates(subset=['customer_idx'])[['customer_idx', 'mean_customer_idx']]

original_sub = original_sub.merge(mean_customer_idx_unique, on='customer_idx', how='left')
original_sub['mean_customer_idx'].fillna(original_train[original_train['customer_idx']=='etc']['mean_customer_idx'].iloc[0],inplace= True)

##################### mean_lead_owner 컬럼 생성 ###################################
original_train['lead_owner_cnt'] = original_train.groupby(['lead_owner'])['is_converted'].transform('count')

#빈도 수 (lead_owner_cnt)가 5 이하인 데이터는 lead_owner를 etc로 통일
n = 5
for i in range(1,n+1):
    original_train.loc[original_train['lead_owner_cnt'] == i,'lead_owner'] = 'etc'

# lead_owner 별 평균 영업 성공률 계산한 mean_lead_owner 추가
original_train['mean_lead_owner'] = original_train.groupby('lead_owner')['is_converted'].transform('mean')
original_train.mean_lead_owner = [converted_mean if original_train.lead_owner.iloc[i] == "etc" else original_train.mean_lead_owner.iloc[i] for i in range(len(original_train))]
mean_customer_idx_unique = original_train.drop_duplicates(subset=['lead_owner'])[['lead_owner', 'mean_lead_owner']]

original_sub = original_sub.merge(mean_customer_idx_unique,on='lead_owner',how='left')
original_sub['mean_lead_owner'].fillna(original_train[original_train['lead_owner']=='etc']['mean_lead_owner'].iloc[0],inplace= True)

##################### corporate_converted_ratio 컬럼 생성 ###################################
table = original_train.value_counts(["response_corporate","is_converted"]).reset_index().pivot(index = "is_converted", columns = "response_corporate", values = 0).T
table["Total"] = table.sum(axis = 1)
table["True_per"] = table[True]/table["Total"]
table = table.fillna(0)
table = table.reset_index()[["response_corporate", "True_per"]].set_index("response_corporate").T.to_dict()
original_train['corporate_converted_ratio'] = [table[i]["True_per"] if i in table.keys() else converted_mean for i in original_train.response_corporate]
original_sub['corporate_converted_ratio'] = [table[i]["True_per"] if i in table.keys() else converted_mean for i in original_sub.response_corporate]


train.mean_customer_idx = original_train.mean_customer_idx
submission.mean_customer_idx = original_sub.mean_customer_idx

train.mean_lead_owner = original_train.mean_lead_owner
submission.mean_lead_owner = original_sub.mean_lead_owner

train.corporate_converted_ratio = original_train.corporate_converted_ratio
submission.corporate_converted_ratio = original_sub.corporate_converted_ratio

### 학습
train = train.drop(['lead_desc_length','expected_timeline','ver_cus',"bant_submit","historical_existing_cnt","id_strategic_ver", "it_strategic_ver","response_corporate","customer_country", "customer_job","customer_position",'ver_win_rate_x','ver_win_ratio_per_bu', "inquiry_type_isna"], axis =1)
submission = submission.drop(['lead_desc_length','expected_timeline','ver_cus',"bant_submit","historical_existing_cnt","id_strategic_ver", "it_strategic_ver","response_corporate","customer_country", "customer_job","customer_position",'ver_win_rate_x','ver_win_ratio_per_bu', "inquiry_type_isna"], axis =1)


# 범주형, 수치형 변수로 나눔
target = ["is_converted"]
categorical = [i for i in train.columns if train[i].dtype == "O"]
numeric = train.columns.drop(categorical + target).tolist()

train["구분"] = "train"
submission["구분"] = "sub"

new = pd.concat([train, submission]).reset_index(drop = True)

# 인코딩
c = []
for i in categorical:
    encoder = OneHotEncoder(drop='first')
    encoder.fit(new[[i]])
    new_df = encoder.transform(new[[i]])
    
    new_df = pd.DataFrame(new_df.toarray(), columns = encoder.get_feature_names_out([i]))
    new = new.drop([i], axis = 1)
    new = pd.concat([new, new_df], axis = 1)
    c += encoder.get_feature_names_out([i]).tolist()

categorical = c
train = new[new.구분 == "train"].drop(["구분"], axis = 1).reset_index(drop = True)
submission = new[new.구분 == "sub"].drop(["구분"], axis = 1).reset_index(drop = True)

train = train.drop(["id"], axis = 1)

scaler = MinMaxScaler()
t = scaler.fit_transform(train[categorical+numeric])
train[categorical+numeric] = pd.DataFrame(t, columns = categorical+numeric)

t = scaler.transform(submission[categorical+numeric])
submission[categorical+numeric] = pd.DataFrame(t, columns = categorical+numeric)

X = train[train.columns.drop(target)]
y = train[target]

x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42, shuffle = True)

smote = SMOTE(random_state=42)
x_train, y_train = smote.fit_resample(x_train, y_train)

best = {'patience': 2,
 'min_delta': 0.004,
 'node1': 16,
 'dropout1': 0.6773681174114016,
 'node2': 16,
 'dropout2': 0.848566979415723,
 'num_layers': 'three',
 'node3': 16,
 'dropout3': 0.6351770236847896,
 'lr': 0.005453416800200536}


tf.random.set_seed(89)
initializer = tf.keras.initializers.HeNormal(seed = 89)

early_stopping = tf.keras.callbacks.EarlyStopping(monitor = 'val_f1_score',
                                                      mode = 'min',
                                                      patience = best["patience"],
                                                      min_delta = best["min_delta"])

    
model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(best["node1"],activation = 'relu', kernel_initializer=initializer))
model.add(tf.keras.layers.Dropout(best["dropout1"]))
model.add(tf.keras.layers.Dense(best["node2"], activation = 'relu', kernel_initializer=initializer))
model.add(tf.keras.layers.Dropout(best["dropout2"]))
model.add(tf.keras.layers.Dense(best["node3"], activation = 'relu', kernel_initializer=initializer))
model.add(tf.keras.layers.Dropout(best["dropout3"]))
model.add(tf.keras.layers.Dense(2, activation = 'softmax', kernel_initializer=initializer))
    
model.compile(loss = tf.keras.losses.BinaryCrossentropy(),
                      optimizer = tf.keras.optimizers.Adam(learning_rate = best["lr"]),
                      metrics = F1Score(num_classes = 2))
    
y_train_one_hot = pd.get_dummies(y_train.is_converted).values.reshape(-1,2)
y_test_one_hot = pd.get_dummies(y_test.is_converted).values.reshape(-1,2)
    
history = model.fit(np.array(x_train).reshape(-1,x_train.shape[1]), y_train_one_hot,
                             epochs = 20,
                             validation_data = [np.array(x_test).reshape(-1,x_train.shape[1]), y_test_one_hot],
                            verbose = 2)
preds = model.predict(submission.drop(["id", "is_converted"], axis = 1).values)
preds = preds[:,1] > 0.5

submission = pd.read_csv("submission.csv")
submission['is_converted'] = preds
submission.to_csv("submission.csv", index=False)
submission.to_csv("submission_mlp.csv", index=False)